In [2]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from src.utils.UsefulPaths import Paths

import torch

In [3]:
paths = Paths()

with open(paths.json_subsectors, 'r') as file:
    subsectors = json.load(file)

In [4]:
df_abstract = pd.read_parquet(paths.raw_parquet_abstract)

offset_div = 50
df_joao = df_abstract.iloc[:offset_div].copy().reset_index(drop='index')
df_leo = df_abstract.iloc[offset_div:offset_div*2].copy().reset_index(drop='index')
df_maxm = df_abstract.iloc[offset_div*2:offset_div*3].copy().reset_index(drop='index')
df_rafael = df_abstract.iloc[offset_div*4:offset_div*5].copy().reset_index(drop='index')
df_thiago = df_abstract.iloc[offset_div*6:offset_div*7].copy().reset_index(drop='index')
df_thiago_many = df_abstract.iloc[offset_div*6:offset_div*32].copy().reset_index(drop='index')

In [15]:

candidate_labels = []
for subsetor_name, subsetor_values in subsectors.items():
    subsetor_definition = subsetor_values.get('Definition', '')
    subsetor_keywords = subsetor_values.get('Keywords', '')
    subsetor_does_include = subsetor_values.get('Does include', '')
    # subsetor_does_not_include = subsetor_values.get('Does not include', '')

    k = f'{subsetor_name}:'

    candidate_labels.append(k)

candidate_labels.append('Other:')
candidate_labels

['Artificial Intelligence, Big Data and Analytics:',
 'Advanced Manufacturing and Robotics:',
 'Cleantech:',
 'Fintech:',
 'Blockchain:',
 'Cybersecurity:',
 'Agtech:',
 'New Food:',
 'Adtech :',
 'Blue Economy:',
 'Digital Media:',
 'Gaming (Digital Media Sub-Cluster):',
 'AR / VR (Digital Media Sub-Cluster):',
 'Edtech:',
 'Industry 4.0 :',
 'Biopharma / Biotech:',
 'Medtech / Medical devices:',
 'Other:']

In [16]:
# # for subsetor_name, subsetor_values in subsectors.items():
# candidate_labels = list(subsectors.keys())
# candidate_labels.append('Other')
# candidate_labels

In [24]:
from transformers import pipeline

# "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
# "facebook/bart-large-mnli"
# "sileod/deberta-v3-base-tasksource-nli"
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

col_names = ['publication_number', 'abstract']
for i in range(1, len(candidate_labels) + 1):
    col_names.append(f'score_{i}')

break_index = 50
sum_of_valid_score = 0

df_all = pd.DataFrame()
for index, row in df_joao.iterrows():
    sequence_to_classify = row['abstract']
    output = classifier(sequence_to_classify, candidate_labels, multi_label=False)

    labels_withou_points = [item.split(':')[0].strip() for item in output['labels']]
    df_scores = pd.DataFrame([output['scores']], columns=labels_withou_points)
    df_scores_sorted = df_scores.iloc[0, :].to_frame().T.sort_values(by=0, axis=1, ascending=False)

    v = list(df_scores_sorted.columns)
    v.insert(0, row['abstract'])
    v.insert(0, row['publication_number'])

    df = pd.DataFrame([v], columns=col_names)

    df_all = pd.concat([df_all, df])

    if df.loc[0, 'score_1'] != 'Other':
        sum_of_valid_score += 1

    # if sum_of_valid_score >= 51:
    #     break
    if break_index == index + 1:
        break

df_all.reset_index(drop='index', inplace=True)

2023-09-01 00:10:05,370 - DEBUG - Resetting dropped connection: huggingface.co
2023-09-01 00:10:05,635 - DEBUG - https://huggingface.co:443 "HEAD /MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/resolve/main/config.json HTTP/1.1" 200 0
C:\Users\thiag\PycharmProjects\Genome\venv\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [26]:
df_all.index = df_all.index + 1
df_all

,publication_number,abstract,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,score_11,score_12,score_13,score_14,score_15,score_16,score_17,score_18
2,20080063564,Embodiments of techniques for determining the ...,Other,Biopharma / Biotech,Medtech / Medical devices,Adtech,Agtech,Fintech,Edtech,Digital Media,Cleantech,AR / VR (Digital Media Sub-Cluster),Blockchain,Industry 4.0,Gaming (Digital Media Sub-Cluster),Blue Economy,New Food,Cybersecurity,"Artificial Intelligence, Big Data and Analytics",Advanced Manufacturing and Robotics
3,20080025285,A method for supporting frequency hopping of a...,Other,Medtech / Medical devices,AR / VR (Digital Media Sub-Cluster),Edtech,Fintech,Agtech,Adtech,Digital Media,Biopharma / Biotech,Industry 4.0,Gaming (Digital Media Sub-Cluster),Blockchain,Cleantech,"Artificial Intelligence, Big Data and Analytics",Blue Economy,New Food,Cybersecurity,Advanced Manufacturing and Robotics
4,20080056857,To correct any positional misalignment of a su...,Other,Medtech / Medical devices,Biopharma / Biotech,Digital Media,Edtech,Gaming (Digital Media Sub-Cluster),Advanced Manufacturing and Robotics,AR / VR (Digital Media Sub-Cluster),Adtech,Industry 4.0,Fintech,Agtech,Blockchain,Cleantech,Blue Economy,Cybersecurity,"Artificial Intelligence, Big Data and Analytics",New Food
5,20080031117,A holographic optical accessing system include...,Other,Medtech / Medical devices,Biopharma / Biotech,Digital Media,AR / VR (Digital Media Sub-Cluster),Gaming (Digital Media Sub-Cluster),"Artificial Intelligence, Big Data and Analytics",Edtech,Fintech,Blue Economy,Adtech,New Food,Agtech,Industry 4.0,Blockchain,Advanced Manufacturing and Robotics,Cleantech,Cybersecurity
6,20080056179,Transmitting an acknowledgement/negative ackno...,Other,Medtech / Medical devices,Digital Media,Edtech,AR / VR (Digital Media Sub-Cluster),Agtech,Adtech,Fintech,Biopharma / Biotech,Cybersecurity,Gaming (Digital Media Sub-Cluster),Industry 4.0,Blockchain,Cleantech,Blue Economy,"Artificial Intelligence, Big Data and Analytics",New Food,Advanced Manufacturing and Robotics
7,20080032266,The present invention is a superstructure body...,Medtech / Medical devices,Biopharma / Biotech,Other,Industry 4.0,Fintech,Advanced Manufacturing and Robotics,AR / VR (Digital Media Sub-Cluster),Edtech,Digital Media,Blockchain,Adtech,"Artificial Intelligence, Big Data and Analytics",Cleantech,Agtech,Gaming (Digital Media Sub-Cluster),Blue Economy,Cybersecurity,New Food
8,20080055749,A fine adjustment mechanism includes a first f...,Medtech / Medical devices,Other,Edtech,Digital Media,Biopharma / Biotech,Fintech,Advanced Manufacturing and Robotics,Adtech,Gaming (Digital Media Sub-Cluster),Agtech,Blockchain,AR / VR (Digital Media Sub-Cluster),Cleantech,Blue Economy,Cybersecurity,Industry 4.0,"Artificial Intelligence, Big Data and Analytics",New Food
9,20080001735,A monitoring system includes one or more wirel...,Other,Medtech / Medical devices,Biopharma / Biotech,Digital Media,Industry 4.0,Cybersecurity,Fintech,Edtech,Gaming (Digital Media Sub-Cluster),Cleantech,Agtech,Adtech,AR / VR (Digital Media Sub-Cluster),Blue Economy,New Food,Blockchain,Advanced Manufacturing and Robotics,"Artificial Intelligence, Big Data and Analytics"
10,20080018979,A method of supplying an electrically controll...,Other,Medtech / Medical devices,Biopharma / Biotech,Cleantech,Industry 4.0,Digital Media,Edtech,Fintech,Adtech,Agtech,AR / VR (Digital Media Sub-Cluster),Blockchain,Gaming (Digital Media Sub-Cluster),Blue Economy,Advanced Manufacturing and Robotics,Cybersecurity,New Food,"Artificial Intelligence, Big Data and Analytics"
11,20080009466,The invention relates to solid crystalline for...,Biopharma / Biotech,Medtech / Medical devices,Other,Edtech,Fintech,Adtech,Agtech,Cleantech,AR / VR (Digital Media Sub-Cluster),Industry 4.0,Blue Economy,Gaming (Digital Media Sub-Cluster),Blockchain,Digital Media,"Artificial Intelligence, Big Data and Analytics",Advanced Manufa

In [22]:
df_all.to_excel(f'C:\\Users\\thiag\\PycharmProjects\\genome\\data\\processed\\class_patent_2.xlsx', index=False, header=True)